In [0]:
%python
#pip install pokemontcgsdk
#dbutils.library.restartPython()

In [0]:
%python
# %pip install pokemontcgsdk
import os
import json
from pokemontcgsdk import Card, RestClient
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    ArrayType,
    IntegerType,
)

# Configure your API key from environment variable
api_key = dbutils.secrets.get(scope="my_scope", key="POKEMON_API_KEY")
if not api_key:
    raise ValueError("API key not found. Set the POKEMON_API_KEY environment variable.")
RestClient.configure(api_key)

# Fetch all cards using the SDK
#cards = list(Card.where(page=1, pageSize=1))
cards = list(Card.all())


# Helper function to recursively convert object attributes to dicts
def obj_to_dict(obj):
    if isinstance(obj, list):
        return [obj_to_dict(item) for item in obj]
    elif hasattr(obj, "__dict__"):
        return {key: obj_to_dict(value) for key, value in obj.__dict__.items()}
    else:
        return obj


# Convert cards to a list of fully serializable dicts
cards_list = [obj_to_dict(c) for c in cards]

# Define the schema explicitly
schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("supertype", StringType(), True),
        StructField("subtypes", ArrayType(StringType()), True),
        StructField("level", StringType(), True),
        StructField("hp", StringType(), True),
        StructField("types", ArrayType(StringType()), True),
        StructField("rules", ArrayType(StringType()), True),
        StructField(
            "ancientTrait",
            StructType(
                [
                    StructField("name", StringType(), True),
                    StructField("text", StringType(), True),
                ]
            ),
            True,
        ),
        StructField(
            "set",
            StructType(
                [
                    StructField("id", StringType(), True),
                    StructField("ptcgoCode", StringType(), True),
                ]
            ),
            True,
        ),
        StructField("number", StringType(), True),
        StructField("artist", StringType(), True),
        StructField("rarity", StringType(), True),
        StructField("flavorText", StringType(), True),
        StructField("nationalPokedexNumbers", ArrayType(IntegerType()), True),
        StructField(
            "images",
            StructType(
                [
                    StructField("small", StringType(), True),
                    StructField("large", StringType(), True),
                ]
            ),
            True,
        ),
    ]
)

# Save the JSON to DBFS using Spark
# spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(cards_list, schema=schema)

# Optional: write to Delta
df.write.format("delta").mode("overwrite").saveAsTable("workspace.pokemon_tcg_collection.tcg_all_cards")
#df.display()

In [0]:
%python
# %pip install pokemontcgsdk
import os
import json
from pokemontcgsdk import Set, RestClient
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    ArrayType,
    IntegerType,
)

# Configure your API key from environment variable
api_key = dbutils.secrets.get(scope="my_scope", key="POKEMON_API_KEY")
if not api_key:
    raise ValueError("API key not found. Set the POKEMON_API_KEY environment variable.")
RestClient.configure(api_key)

# Fetch all cards using the SDK
# cards = list(Card.where(page=1, pageSize=1))
# sets = list(sets = Set.where(page=2, pageSize=10))
#sets = Set.where(page=2, pageSize=10)
sets = Set.all()


# Helper function to recursively convert object attributes to dicts
def obj_to_dict(obj):
    if isinstance(obj, list):
        return [obj_to_dict(item) for item in obj]
    elif hasattr(obj, "__dict__"):
        return {key: obj_to_dict(value) for key, value in obj.__dict__.items()}
    else:
        return obj


# Convert cards to a list of fully serializable dicts
sets_list = [obj_to_dict(s) for s in sets]

# Define the schema explicitly
schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("series", StringType(), True),
        StructField("printedTotal", IntegerType(), True),
        StructField("total", IntegerType(), True),
        StructField("ptcgoCode", StringType(), True),
        StructField("releaseDate", StringType(), True),
        StructField("updatedAt", StringType(), True),
        StructField(
            "images",
            StructType(
                [
                    StructField("symbol", StringType(), True),
                    StructField("logo", StringType(), True),
                ]
            ),
            True,
        ),
    ]
)

# Save the JSON to DBFS using Spark
# spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(sets_list, schema=schema)

# Optional: write to Delta
df.write.format("delta").mode("overwrite").saveAsTable("workspace.pokemon_tcg_collection.tcg_all_sets")
#df.display()

In [0]:
	WITH source AS (
  SELECT
		t1.Quantity
		,t1.Name
		,t1.Simple_Name
		,t1.Set
		,t1.Card_Number
		,CASE
			WHEN
				TRY_CAST(ELEMENT_AT(SPLIT(t1.Card_Number, '/'), 1) AS INT) IS NOT NULL
			THEN
				CAST(TRY_CAST(ELEMENT_AT(SPLIT(t1.Card_Number, '/'), 1) AS INT) AS STRING)
			ELSE ELEMENT_AT(SPLIT(t1.Card_Number, '/'), 1)
		END AS Print_Number
		,TRY_ELEMENT_AT(SPLIT(t1.Card_Number, '/'), 2) AS Set_Total
		,(
			CASE
				WHEN t1.Set_Code LIKE "BS2" THEN "B2"
				ELSE t1.Set_Code
			END
		) AS Set_Code
		,t1.Printing
		,t1.Condition
		,t1.Language
		,t1.Rarity
		,t1.Product_ID
		,t1.SKU
		,t1.Price
		,t1.Price_Each


	FROM
		workspace.pokemon_tcg_collection.tcg_collection AS t1
  )
--SELECT * FROM source;

SELECT 
    t1.*
    ,t2.id
		,t2.ptcgoCode
    FROM source AS t1
			LEFT JOIN workspace.pokemon_tcg_collection.tcg_all_sets AS t2
				ON t1.Set_Code = t2.ptcgoCode
-- SELECT * FROM workspace.pokemon_tcg_collection.tcg_all_sets;
--SELECT * FROM workspace.pokemon_tcg_collection.tcg_all_cards WHERE name LIKE "Braixen"

In [0]:
%sql
CREATE OR REPLACE TABLE pokemon_tcg_collection.bronze.tcg_collection
SELECT
	Quantity
	,Name
	,Simple_Name
	,Set
	,Card_Number
	,Set_Code
	,CASE WHEN Printing LIKE "1st%" THEN "Unlimited" ELSE Printing END AS Printing
	,Condition
	,Language
	,Rarity
	,Product_ID
	,SKU
	,Price
	,Price_Each

FROM
	pokemon_tcg_collection.bronze.tcg_collection

In [0]:
%sql
SELECT * FROM pokemon_tcg_collection.bronze.tcg_all_cards;

In [0]:
%sql
SELECT * FROM pokemon_tcg_collection.bronze.tcg_all_sets